In [1]:
from bs4 import BeautifulSoup as BS
from urllib.request import urlopen, Request
import pandas as pd
import requests
import re

/Users/shrikirti/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from bs4 import BeautifulSoup
import requests

#list of city url's

list_of_Alameda_county = [
    'https://www.mlslistings.com/Search/Result/4e0db75f-6859-49ba-b043-39ac43aa350a/1',
    'https://www.mlslistings.com/Search/Result/4e0db75f-6859-49ba-b043-39ac43aa350a/2'
    
]

def get_city_html(list_of_Alameda_county):
    cities = []
    for url in list_of_Alameda_county:
        response = requests.get(url)
        if response.status_code == 200:
            soup_mls = BeautifulSoup(response.content, 'html.parser')
            list_of_home_of_one_city  = soup_mls.find_all("div", attrs={'class': "col-xs-12 mt-1 mb-0 px-0 card-group listing-card clearfix pos-relative"})
            cities = cities+list_of_home_of_one_city
            
        else:
            print(f"Failed to fetch data from URL: {url}")
    return cities

homes_list = get_city_html(list_of_Alameda_county)
print(len(homes_list))

60


In [4]:
df_mls = pd.DataFrame(columns=['MLS_id','url','No_of_days_onsite'])
idx=0
for url in homes_list:
    a = url.find("a", attrs={'class':"search-nav-link"})
    df_mls.loc[idx, 'url'] = "https://www.mlslistings.com" + a['href']
    df_mls.loc[idx, 'MLS_id'] = url.find("strong", attrs={'class':"info-item-label"}).get_text(strip=True)
    df_mls.loc[idx, 'No_of_days_onsite'] = url.find("span", attrs={'class':"listing-dom-block pull-left pl-25"}).get_text(strip=True)

    idx += 1
print(df_mls)

         MLS_id                                                url  \
0    BE41057568  https://www.mlslistings.com/property/be4105756...   
1    ML81964819  https://www.mlslistings.com/property/ml8196481...   
2    ML81964808  https://www.mlslistings.com/property/ml8196480...   
3    BE41058854  https://www.mlslistings.com/property/be4105885...   
4    ML81964703  https://www.mlslistings.com/property/ml8196470...   
5    BE41058800  https://www.mlslistings.com/property/be4105880...   
6    BE41056362  https://www.mlslistings.com/property/be4105636...   
7    BE41058790  https://www.mlslistings.com/property/be4105879...   
8    ML81964614  https://www.mlslistings.com/property/ml8196461...   
9    EB41058804  https://www.mlslistings.com/property/eb4105880...   
10   BE41058778  https://www.mlslistings.com/property/be4105877...   
11   EB41058817  https://www.mlslistings.com/property/eb4105881...   
12   BE41056473  https://www.mlslistings.com/property/be4105647...   
13   BE41056500  htt

In [5]:
list_links_mls = df_mls['url'].tolist()

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [7]:
headers={
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
}
request_list=[]

with requests.session() as s:
  for link in list_links_mls:
    request = s.get(link, params={'headers': headers})
    request_list.append(request)

soup_list = []
for req in request_list:
  soup = BeautifulSoup(req.content, 'html.parser')
  #print(soup.title)
  soup_list.append(soup)

In [9]:
from bs4 import BeautifulSoup
import pandas as pd

data = {"MLS_id":[],'Price': [],'Beds': [], 'Baths': [], 'Sqft': [], 
        'Year_built': [], 'Location': [],'Property_type':[] ,'HOA':[]}
HOA_value = 0
for sp in soup_list:
    h4_price = sp.find("h4", attrs={'class':"font-weight-bold h font-size-xl line-height-lg"})
    h4_price_text = h4_price.get_text(separator=' ', strip=True)
    price = h4_price_text.split(" ")[0]


    table = sp.find("table", attrs={'class':"table table-sm text-xs-center facts-table mb-1"})
    columns = table.find('thead').find_all('th')
    Beds = (columns[0].get_text(strip=True))
    Baths = (columns[1].get_text(strip=True))
    Sqft =  (columns[2].get_text(strip=True))
    Year_built = (columns[4].get_text(strip=True))
    mls_id_p = sp.find("p", attrs={'class':"card-text font-size-midr line-height-xl"})
    mls_id = mls_id_p.get_text(strip=True)
    location = sp.find("h2", attrs={'class':"font-weight-bold h font-size-xl line-height-lg"})
    loc = location.get_text(strip=True)
    pr_type = sp.find("small", attrs={'class':"font-size-base sm-sticky"})
    property = pr_type.get_text(strip=True)
    hoa_p = sp.find_all("p", attrs={'class':"card-text font-size-midr line-height-xl"})
    for tag in hoa_p:
      tag_text = str(tag.contents)
      hoapattern = r'\$(\d+)'
      HOA_match = re.search(hoapattern, tag_text)
      if HOA_match:
        HOA_value = HOA_match.group(1)
    
   
    data['Beds'].append(Beds)
    data['Baths'].append(Baths)
    data['Sqft'].append(Sqft)
    data['Year_built'].append(Year_built)
    data['Price'].append(price)
    data['MLS_id'].append(mls_id)
    data['Location'].append(loc)
    data['Property_type'].append(property)
    data['HOA'].append(HOA_value)
              
df = pd.DataFrame(data)

merged_df = pd.merge(df, df_mls[['MLS_id', 'No_of_days_onsite']], on='MLS_id', how='left')

print(merged_df)
merged_df.to_csv('Final_part.csv', index=False)

         MLS_id       Price Beds Baths   Sqft Year_built  \
0    BE41057568    $725,000    2     1  1,134       1929   
1    ML81964819  $1,649,900    4     3  2,701       2017   
2    ML81964808    $739,000    3     1  1,023       1949   
3    BE41058854    $725,000    3     2  1,000       1955   
4    ML81964703  $2,750,000    –     –      –          –   
5    BE41058800    $579,800    2   1/1  1,110       1987   
6    BE41056362    $898,000    3     2  1,107       1957   
7    BE41058790    $839,000    3     2  1,367       1955   
8    ML81964614    $685,000    2     2  1,393       1992   
9    EB41058804  $1,149,000    4     2  1,849       1977   
10   BE41058778    $615,000    2     1    853       1951   
11   EB41058817    $649,000    2   2/1  1,270       1973   
12   BE41056473    $999,999    3     2  2,091       1958   
13   BE41056500    $799,950    3     1  1,025       1951   
14   BE41058587    $878,000    3     2  1,138       1955   
15   BE41058751    $599,000    3     2  